<a href="https://colab.research.google.com/github/sriharshamutnuru/AI_Learning/blob/main/Day9_COT_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# 📘 Day 9 — Chain-of-Thought & Structured Outputs
# ============================================================

!pip install --quiet openai==1.109.1 jsonschema

from openai import OpenAI
from google.colab import userdata
import json
from jsonschema import validate, ValidationError

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

MODEL = "gpt-4o-mini"

# --- Step 1: Prompt with CoT + Structured Output ---
prompt = """
You are a reasoning assistant.
Solve the following problem step-by-step, and return the final result as valid JSON.

Problem: A factory produces 1200 units a day. 10% are defective.
Next day, production increases by 20%, but defect rate drops to 5%.
Calculate how many non-defective units were produced on the second day.

Return JSON in this format:
{
  "reasoning": "string",
  "non_defective_units": number
}

Return only JSON, no other text.
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful reasoning assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3
)

output = response.choices[0].message.content.strip()

print("🧠 Model Output:")
print(output)

# --- Step 2: Validate JSON Schema ---
schema = {
    "type": "object",
    "properties": {
        "reasoning": {"type": "string"},
        "non_defective_units": {"type": "number"}
    },
    "required": ["reasoning", "non_defective_units"]
}

try:
    parsed_output = json.loads(output)
    validate(instance=parsed_output, schema=schema)
    print("\n✅ JSON is valid and schema-compliant!")
    print(f"Non-defective units: {parsed_output['non_defective_units']}")
except (json.JSONDecodeError, ValidationError) as e:
    print("\n❌ JSON validation failed!")
    print("Error:", e)


🧠 Model Output:
{
  "reasoning": "On the first day, the factory produces 1200 units with a defect rate of 10%. Therefore, the number of defective units is 10% of 1200, which is 120 units. The number of non-defective units produced on the first day is 1200 - 120 = 1080 units. On the second day, production increases by 20%, so the new production amount is 1200 * 1.20 = 1440 units. The defect rate drops to 5%, so the number of defective units on the second day is 5% of 1440, which is 72 units. Therefore, the number of non-defective units produced on the second day is 1440 - 72 = 1368 units.",
  "non_defective_units": 1368
}

✅ JSON is valid and schema-compliant!
Non-defective units: 1368
